# list 라벨링의 오류 파헤쳐보기

- 1300개의 이미지를 2진 분류하기 위해 라벨링을 \[0, 1\], \[1, 0\]로 하였을 때 Accuracy, Precision, Recall 값을 살펴보자

## Step 1. 실험환경 set-up

#### 라이브러리 실행

In [1]:
from tensorflow.keras             import utils, regularizers
from tensorflow.keras.models      import Sequential, load_model
from tensorflow.keras.layers      import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers  import Adam
from tensorflow.keras.constraints import MaxNorm
from sklearn.model_selection      import train_test_split  # 데이터 전처리에 필요한 패키지

import re    # 정규표현식 관련된 작업에 필요한 패키지
import os    # I/O 관련된 작업에 필요한 패키지 
import pandas as pd     # 데이터 전처리 관련된 작업에 필요한 패키지
import numpy as np      # 데이터 array 작업에 필요한 패키지
import tensorflow as tf  # 딥러닝 관련된 작업에 필요한 패키지
import matplotlib.pyplot as plt    # 데이터 시각화에 관련된 작업에 필요한 패키지
from PIL import Image

#### 필요한 변수들 생성

In [2]:
# 데이터 로드할 때 빠르게 로드할 수 있도록하는 설정 변수
AUTOTUNE = tf.data.experimental.AUTOTUNE

# BATCH_SIZE 변수
BATCH_SIZE = 32

# 이미지 사이즈 변수
IMAGE_SIZE = [256, 256]

# EPOCH 크기 변수
EPOCHS = 3

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Step 2. 데이터 준비하기


#### 데이터의 개수 확인

In [4]:
# 데이터 ROOT 경로 변수
ROOT_PATH = os.path.join(os.getenv('HOME') + '/CellPin_Trial')
print(ROOT_PATH)

train_filenames = tf.io.gfile.glob(str(ROOT_PATH + '/CellPin/train/*/*'))
test_filenames = tf.io.gfile.glob(str(ROOT_PATH + '/CellPin/test/*/*'))
val_filenames = tf.io.gfile.glob(str(ROOT_PATH + '/CellPin/val/*/*'))

print(len(train_filenames))
print(len(test_filenames))
print(len(val_filenames))

/home/aiffel-dj63/CellPin_Trial
780
260
260


#### train 데이터 안에 정상 이미지 수와 CPE 이미지 수

In [5]:
COUNT_NORMAL = len([filename for filename in train_filenames if "normal" in filename])
print("NORMAL images count in training set: " + str(COUNT_NORMAL))

COUNT_CPE = len([filename for filename in train_filenames if "cpe" in filename])
print("CPE images count in training set: " + str(COUNT_CPE))

NORMAL images count in training set: 429
CPE images count in training set: 351


#### val 데이터 안에 정상 이미지 수와 CPE 이미지 수

In [6]:
COUNT_NORMAL = len([filename for filename in val_filenames if "normal" in filename])
print("NORMAL images count in training set: " + str(COUNT_NORMAL))

COUNT_CPE = len([filename for filename in val_filenames if "cpe" in filename])
print("CPE images count in training set: " + str(COUNT_CPE))

NORMAL images count in training set: 143
CPE images count in training set: 117


#### test 데이터 안에 정상 이미지 수와 CPE 이미지 수

In [7]:
COUNT_NORMAL = len([filename for filename in test_filenames if "normal" in filename])
print("NORMAL images count in training set: " + str(COUNT_NORMAL))

COUNT_CPE = len([filename for filename in test_filenames if "cpe" in filename])
print("CPE images count in training set: " + str(COUNT_CPE))

NORMAL images count in training set: 143
CPE images count in training set: 117


#### tf.data 인스턴스를 만들기

In [8]:
train_list_ds = tf.data.Dataset.from_tensor_slices(train_filenames)
val_list_ds = tf.data.Dataset.from_tensor_slices(val_filenames)

#### train data와 val data의 개수 확인

In [9]:
TRAIN_IMG_COUNT = tf.data.experimental.cardinality(train_list_ds).numpy()
print("Training images count: " + str(TRAIN_IMG_COUNT))

VAL_IMG_COUNT = tf.data.experimental.cardinality(val_list_ds).numpy()
print("Validating images count: " + str(VAL_IMG_COUNT))

Training images count: 780
Validating images count: 260


#### 라벨 이름들 확인

In [10]:
CLASS_NAMES = np.array([str(tf.strings.split(item, os.path.sep)[-1].numpy())[2:-1]
                        for item in tf.io.gfile.glob(str(ROOT_PATH + "/CellPin/train/*"))])
print(CLASS_NAMES)

['normal' 'cpe']


#### 라벨링 함수를 만들어 보기

In [11]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2] == "cpe"

#### decode_img 함수와 process_path 함수를 만들기

In [12]:
def decode_img(img):
  # 이미지를 uint8 tensor로 바꾼다.
  img = tf.image.decode_jpeg(img, channels=3)
  # img를 범위 [0,1]의 float32 데이터 타입으로 바꾼다.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # img의 이미지 사이즈를 IMAGE_SIZE에서 지정한 사이즈로 수정한다.
  return tf.image.resize(img, IMAGE_SIZE)

def process_path(file_path):
    label = get_label(file_path)
    if label == True:
        label = ([0, 1])
    else:
        label = ([1, 0])
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

#### train과 val에 함수들 적용해주기

In [13]:
train_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


#### 전처리 함수 만들기 및 적용

In [14]:
def prepare_for_training(ds, shuffle_buffer_size=1000):
    
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    ds = ds.repeat()

    ds = ds.batch(BATCH_SIZE)

    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds

train_ds = prepare_for_training(train_ds)
val_ds = prepare_for_training(val_ds)

#### Image shape를 변경 체크하기

In [15]:
for image, label in train_ds.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

Image shape:  (32, 256, 256, 3)
Label:  [[0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]]


#### test 데이터 셋도 만들어 보기 & 데이터의 갯수 확인

In [16]:
test_list_ds = tf.data.Dataset.list_files(str(ROOT_PATH + '/CellPin/test/*/*'))
TEST_IMAGE_COUNT = tf.data.experimental.cardinality(test_list_ds).numpy()
test_ds = test_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE)

print(TEST_IMAGE_COUNT)

260


#### model building

In [17]:
vgg_layer = tf.keras.applications.VGG19(include_top = False,
                                        weights = "imagenet",
                                        input_shape = (256, 256, 3),
                                        classes = 2,
                                        classifier_activation = "sigmoid")

fc_layer = [
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(4096),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(4096),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(2),
    tf.keras.layers.Activation('sigmoid')
]

vgg_layer.trainable = False

vgg19 = tf.keras.models.Sequential([vgg_layer] + fc_layer)

## Step 6. 모델 훈련

#### GPU사용하여 model complie

In [18]:
with tf.device('/GPU:0'):
    METRICS = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.FalseNegatives(name='FN'),
        tf.keras.metrics.FalsePositives(name='FP'),
        tf.keras.metrics.TrueNegatives(name='TN'),
        tf.keras.metrics.TruePositives(name='TP')
    ]
    
    vgg19.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=METRICS
    )
    
    # early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 2, restore_best_weights = True)
    check_point = tf.keras.callbacks.ModelCheckpoint(ROOT_PATH + '/Metrics_VGG19_with_matrics_2.h5')

#### 모델 fitting

In [19]:
with tf.device('/GPU:0'):
    history = vgg19.fit(
        train_ds,
        steps_per_epoch=TRAIN_IMG_COUNT // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=val_ds,
        validation_steps=VAL_IMG_COUNT // BATCH_SIZE,
        callbacks=[check_point]
    )

Epoch 1/3
24/24 [==============================] - 84s 2s/step - loss: 7.1882 - accuracy: 0.5802 - precision: 0.5718 - recall: 0.5522 - FN: 169.4800 - FP: 156.1200 - TN: 258.6000 - TP: 245.2400 - val_loss: 0.3352 - val_accuracy: 0.8516 - val_precision: 0.8417 - val_recall: 0.8516 - val_FN: 38.0000 - val_FP: 41.0000 - val_TN: 215.0000 - val_TP: 218.0000
Epoch 2/3
24/24 [==============================] - 45s 2s/step - loss: 0.3942 - accuracy: 0.8610 - precision: 0.8517 - recall: 0.8540 - FN: 63.4400 - FP: 61.7600 - TN: 352.9600 - TP: 351.2800 - val_loss: 0.2961 - val_accuracy: 0.8750 - val_precision: 0.8845 - val_recall: 0.8672 - val_FN: 34.0000 - val_FP: 29.0000 - val_TN: 227.0000 - val_TP: 222.0000
Epoch 3/3
24/24 [==============================] - 46s 2s/step - loss: 0.2979 - accuracy: 0.8838 - precision: 0.8865 - recall: 0.8854 - FN: 47.6400 - FP: 48.4800 - TN: 366.2400 - TP: 367.0800 - val_loss: 0.2319 - val_accuracy: 0.9062 - val_precision: 0.9098 - val_recall: 0.9062 - val_FN: 24.

## Step 8. 최종 Test dataset 검정하기

In [20]:
model = tf.keras.models.load_model(ROOT_PATH + '/Metrics_VGG19_with_matrics_2.h5')
loss, acc, prec, rec, fn, fp, tn, tp = model.evaluate(test_ds)

9/9 [==============================] - 20s 2s/step - loss: 0.2200 - accuracy: 0.9192 - precision: 0.9192 - recall: 0.9192 - FN: 21.0000 - FP: 21.0000 - TN: 239.0000 - TP: 239.0000


> accuracy와 precision, recall이 모두 동일하게 나왔다. 심지어 FN = FP, TN = TP로 같다고 나왔다.


## CPE의 이미지 전부를 예측해보기

In [21]:
test_list_ds1 = tf.data.Dataset.list_files(str(ROOT_PATH + '/CellPin/test/cpe/*'))
TEST_IMAGE_COUNT1 = tf.data.experimental.cardinality(test_list_ds1).numpy()
test_ds1 = test_list_ds1.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds1 = test_ds1.batch(BATCH_SIZE)

print(TEST_IMAGE_COUNT1)

117


In [22]:
i = 0

for predict in model.predict(test_ds1):
    if np.argmax(predict) == 1:
        print(predict, np.argmax(predict), 'CPE 입니다.')
    else: 
        print(predict, np.argmax(predict), 'Normal 입니다.')
        i += 1
        
print(i)

[0.00418973 0.99503064] 1 CPE 입니다.
[0.83785665 0.1412478 ] 0 Normal 입니다.
[0.02010306 0.97925013] 1 CPE 입니다.
[0.0040025 0.9953348] 1 CPE 입니다.
[0.00872996 0.9905405 ] 1 CPE 입니다.
[0.00775086 0.99160624] 1 CPE 입니다.
[0.07357534 0.9297622 ] 1 CPE 입니다.
[0.2631793 0.7452904] 1 CPE 입니다.
[0.00528756 0.99369377] 1 CPE 입니다.
[0.00116666 0.9984584 ] 1 CPE 입니다.
[0.48065075 0.52049094] 1 CPE 입니다.
[0.00451274 0.9948549 ] 1 CPE 입니다.
[0.00841645 0.9910295 ] 1 CPE 입니다.
[0.00212317 0.99748105] 1 CPE 입니다.
[0.02331212 0.9763743 ] 1 CPE 입니다.
[0.05252776 0.949476  ] 1 CPE 입니다.
[0.00779171 0.9912872 ] 1 CPE 입니다.
[0.17573255 0.8310337 ] 1 CPE 입니다.
[0.01626938 0.9819919 ] 1 CPE 입니다.
[0.04478972 0.9558738 ] 1 CPE 입니다.
[0.16925026 0.8369431 ] 1 CPE 입니다.
[0.02108128 0.978949  ] 1 CPE 입니다.
[0.8064055  0.17592742] 0 Normal 입니다.
[0.04431237 0.9565291 ] 1 CPE 입니다.
[0.03263175 0.966462  ] 1 CPE 입니다.
[0.00640087 0.99302155] 1 CPE 입니다.
[0.13451205 0.8685369 ] 1 CPE 입니다.
[0.02725004 0.97245693] 1 CPE 입니다.
[0.00896167 0.9904

> CPE 이미지 117개 중 107개를 CPE로 예측, 10개를 Normal로 예측

## Normal 이미지를 전부 예측해보기

In [23]:
test_list_ds2 = tf.data.Dataset.list_files(str(ROOT_PATH + '/CellPin/test/normal/*'))
TEST_IMAGE_COUNT2 = tf.data.experimental.cardinality(test_list_ds2).numpy()
test_ds2 = test_list_ds2.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds2 = test_ds2.batch(BATCH_SIZE)

print(TEST_IMAGE_COUNT2)

143


In [24]:
i = 0

for predict in model.predict(test_ds2):
    if np.argmax(predict) == 1:
        print(predict, np.argmax(predict), 'CPE 입니다.')
        i += 1
    else: 
        print(predict, np.argmax(predict), 'Normal 입니다.')

print(i)

[0.9059703  0.07792253] 0 Normal 입니다.
[0.39344832 0.5989817 ] 1 CPE 입니다.
[0.8931714  0.09001957] 0 Normal 입니다.
[0.832109   0.15146664] 0 Normal 입니다.
[0.89518535 0.08821782] 0 Normal 입니다.
[0.94115716 0.04550407] 0 Normal 입니다.
[0.8484046  0.13124318] 0 Normal 입니다.
[0.5422126  0.46594265] 0 Normal 입니다.
[0.74163973 0.24520913] 0 Normal 입니다.
[0.59547675 0.4155504 ] 0 Normal 입니다.
[0.9164487 0.0685638] 0 Normal 입니다.
[0.842943   0.13609105] 0 Normal 입니다.
[0.95098203 0.03714918] 0 Normal 입니다.
[0.96065176 0.02902253] 0 Normal 입니다.
[0.940238   0.04661422] 0 Normal 입니다.
[0.8563945 0.1276849] 0 Normal 입니다.
[0.68802136 0.30948544] 0 Normal 입니다.
[0.957892   0.03135256] 0 Normal 입니다.
[0.800417  0.1809734] 0 Normal 입니다.
[0.04438325 0.95697516] 1 CPE 입니다.
[0.961995   0.02785412] 0 Normal 입니다.
[0.9222874  0.06188171] 0 Normal 입니다.
[0.87704635 0.10337438] 0 Normal 입니다.
[0.47112614 0.53264403] 1 CPE 입니다.
[0.9311699  0.05566593] 0 Normal 입니다.
[0.91899574 0.06522862] 0 Normal 입니다.
[0.89681995 0.08531622] 0 N

> Normal 이미지 143개 중 11개를 CPE로 132개를 Normal로 예측하였다.

# Discussion

TP = TN = 239 (이는 위에서 TP + TN의 값이다.)
FN = FP = 21 (이는 위에서 FN+ FP의 값이다.)

위와 같은 결과가 나왔는데 TP와 TN을 더한 값으로 TP와 TN이 동일하게 나왔고, 마찬가지로 FN과 FP를 더한 값으로 FN과 FP이 동일하게 나왔다. 이는 위에서 보았듯이 Accuracy와 Precision, Recall이 동일하게 나오게 되는 조건을 만족하게 되는 것이다.

 

즉, 해석해보자면 라벨링을 리스트(\[1, 0\], \[0, 1\])로 해준 경우 맞춘 것(TP, TN)과 틀린 것(FN, FP)으로만 구별할 수 있는 듯하다.